In [ ]:
# https://www.geeksforgeeks.org/python-sqlite-creating-a-new-database/#

from sqlalchemy import create_engine, func

import matplotlib.pyplot as plt

from sqlalchemy.orm import declarative_base, Session, relationship

from sqlalchemy import Column, Integer, String, Float, DateTime, ForeignKey

from sqlalchemy.schema import ForeignKeyConstraint

import sqlite3 

import pandas as pd

from datetime import datetime

In [ ]:
df = pd.read_csv("Exports/cleaned.csv")
df.head()

In [ ]:
market_df = df[["country", "mkt_name", "ISO3.1", "currency"]]
market_df.drop_duplicates(inplace=True)
market_df.rename(columns={"mkt_name": "market", "ISO3.1": "code"}, inplace=True)
market_df

In [ ]:
market_df.columns

In [ ]:
food_prices_db = "food_prices.sqlite"
  
try: 
  conn = sqlite3.connect(food_prices_db) 
  print(f"Database {food_prices_db} formed.") 
except: 
  print(f"Database {food_prices_db} not formed.")

In [ ]:
Base = declarative_base()

In [ ]:
class Market(Base):
    __tablename__ = 'markets'
    country = Column(String(50), primary_key=True)
    market_name = Column(String(50), primary_key=True)
    code = Column(String(3))
    currency = Column(String(3))
    food_products = relationship("Food", back_populates="market")

In [ ]:
class Food(Base):
    __tablename__ = 'food_prices'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(20))
    date = Column(DateTime)
    open = Column(Float)
    close = Column(Float)
    inflation = Column(Float)
    trust = Column(Float)
    country = Column(String(50))
    market_name = Column(String(50))
    
    __table_args__ = (
        ForeignKeyConstraint(
            ['country', 'market_name'],
            ['markets.country', 'markets.market_name']
        ),
    )
    
    market = relationship("Market", back_populates="food_products")

In [ ]:
engine = create_engine("sqlite:///food_prices.sqlite")
conn = engine.connect()

In [ ]:
# Drop all tables before creating to prevent duplicates
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [ ]:
# Insert data into the market table
for index, row in market_df.iterrows():
    market = Market(
        country= row["country"],
        market_name = row["market"], 
        code= row["code"],
        currency = row["currency"]
    )
    session.add(market)

In [ ]:
# Insert data into the food table
food_df = df.drop(columns=["ISO3", "ISO3.1", "currency", "data_coverage", "data_coverage_recent", "index_confidence_score", "spatially_interpolated", "o_food_price_index", "c_food_price_index", "trust_food_price_index"], axis=1)
for index, row in food_df.iterrows():
    row = row.dropna()
    country = row["country"]
    market_name = row["mkt_name"]
    date = datetime.strptime(row["DATES"], "%Y-%m-%d")

    food_data = {}

    print(f"Inserting {market_name} of {country} {date} data...")
    # Extract food name from the column names using the .startswith method and assign data using a dictionary
    for index, value in row.items():
        if index.startswith("o_"):
            food = index.split("_")[-1]
            if food not in food_data:
                food_data[food] = {}
            food_data[food]["open"] = value
        elif index.startswith("c_"):
            food_data[food]["close"] = value
        elif index.startswith("trust_"):
            food_data[food]["trust"] = value
        elif index.startswith("inflation_"):
            food_data[food]["inflation"] = value


    for name, items in food_data.items():
        open_value = items.get("open", None)
        close_value = items.get("close", None)
        inflation_value = items.get("inflation", None)
        trust_value = items.get("trust", None)

        food = Food(
            name=name,
            date=date,
            open=open_value,
            close=close_value,
            inflation=inflation_value,
            trust=trust_value,
            country=country,
            market_name=market_name,
        )

        try:
            session.add(food)
        except:
            print("Error encountered!")
        else:
            print(f"Successfully added {name} data!")

In [ ]:
# Commit changes
session.commit()

In [ ]:
# Query market names
market_list = session.query(Market)
for market in market_list:
    print(market.market_name)

In [ ]:
# Query food names
food_list = session.query(Food)
for food in food_list:
    print(food.name)

In [ ]:
# Query Afghanistan food data
market = session.query(Market).filter_by(country="Afghanistan").first()

# Access related food items
for food in market.food_products:
    print(f"Food Name: {food.name}, Date: {food.date}, Open Price: {food.open}, Close Price: {food.close}, Trust: {food.trust}, Inflation: {food.inflation}")

In [ ]:
# Query the average open, close, inflation, and trust
sel = [Food.name,
       func.avg(Food.open),
       func.avg(Food.close),
       func.avg(Food.inflation),
       func.avg(Food.trust)]
april_averages = (
    session.query(*sel)
    .filter(
        Food.market.has(country="Afghanistan", market_name="Badakhshan"),  
        func.strftime("%m", Food.date) == "04" 
    )
    .group_by(Food.name)
    .order_by(Food.name)
    .all()
)

In [ ]:
# Graphing the results
april_df = pd.DataFrame(april_averages, columns=['Food', 'open_avg', 'close_avg', 'inflation_avg', 'trust_avg'])
april_df.set_index('Food', inplace=True)
ax = april_df.plot.bar(figsize=(16, 9))
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f')
plt.xticks(rotation=0)
plt.title("Average April Food Prices in Badakhsan, Afghanistan")
plt.tight_layout()
plt.show()

In [ ]:
#close session
conn.close()